<a href="https://colab.research.google.com/github/chenyicheng2008/google_drive/blob/main/ollama_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

# 1. 服務帳號認證
gauth = GoogleAuth()
# 使用 JSON 檔案進行非互動式認證 (Service Account)
# 正確設定 client_config 以供 ServiceAuth 使用
gauth.settings['client_config'] = {
    'service_account': {
        'client_json_file_path': 'client_secrets.json' # 確保此檔案是您的服務帳號金鑰檔案
    }
}
gauth.settings['oauth_scope'] = ['https://www.googleapis.com/auth/drive']
gauth.settings['save_credentials'] = False # 不儲存憑證
gauth.ServiceAuth()

drive = GoogleDrive(gauth)





#整段程式註解
'''
# 2. 下載檔案 (使用檔案ID)
file_id = 'YOUR_FILE_ID_ON_DRIVE'
file = drive.CreateFile({'id': file_id})
file.GetContentFile('downloaded_file.txt')
print(f"File {file['title']} downloaded successfully.")
'''

In [ ]:
import sys
# 確保 Drive 已掛載

# 將安裝路徑加入系統路徑，這樣 Python 才能找到 yfinance
sys.path.insert(0, '/content/drive/My Drive/colab_packages')

In [ ]:
DOC_ID_gmail = '1Q861W26L2Z6hIcIqVIi7NDkev22uJn1_NdNI_sY-vuw'
gmail_txt = r'/content/drive/MyDrive/LINE_mail_backup2008Gmail/EmailBackup.txt'
Line_txt = r'/content/drive/MyDrive/LINE_mail_backup2008Gmail/LineBackup.txt'

DOC_ID_line = "1VhKPn9kZ2RYVZfQckV4AIaMB3xS_gd14TQU1K-fPol4"

In [ ]:
#read DOC_ID_gmail_dir
with open(gmail_txt, 'r') as f:
    gmail_file = f.read()
with open(Line_txt, 'r') as f:
    line_file = f.read()

In [ ]:
#!ollama pull gpt-oss:120b-cloud
!ollama signin

/bin/bash: line 1: ollama: command not found


In [ ]:
from google.colab import auth
from googleapiclient.discovery import build

# Authenticate to Google Colab
auth.authenticate_user()

# Initialize the Google Docs service client
docs_service = build('docs', 'v1')

print("Google Colab authenticated and Docs service initialized.")

Google Colab authenticated and Docs service initialized.


In [ ]:
pip install tiktoken

In [ ]:
import tiktoken

def count_tokens(text, model_name="gpt-4"):
    """Counts the number of tokens in a text using tiktoken."""
    encoding = tiktoken.encoding_for_model(model_name)
    tokens = encoding.encode(text)
    return len(tokens)

# Assuming gmail_file and line_file are already loaded from previous cells
gmail_tokens = count_tokens(gmail_file)
line_tokens = count_tokens(line_file)

print(f"Gmail file token count: {gmail_tokens}")
print(f"Line file token count: {line_tokens}")

Gmail file token count: 63153
Line file token count: 51066


## Construct Summarization Prompt

### Subtask:
Create a detailed prompt for the Ollama model, instructing it to summarize '近3日重大個股及產業訊息及新聞,及未來一個月會議排程' from the truncated combined document content.


**Reasoning**:
To construct the detailed prompt for the Ollama model as instructed, I will define the `overall_messages` variable with the specified structure, combining the summarization request and the truncated document content.



In [ ]:
combined_document_content = gmail_file + line_file
#combined_document_content_truncated = combined_document_content[:200000]
overall_messages = [
    {
        'role': 'user',
        'content': "Summarize '近5日股票市場個股及產業訊息及新聞,及今日及未來一個月會議排程, 列示舉辦單位及地點' from the following content: " + combined_document_content,
    },
]

print("Prompt created successfully.")

Prompt created successfully.


In [ ]:
import os
from ollama import Client, ResponseError
import requests # Added for robust error handling potential within the try-except

model_to_use = 'gpt-oss:120b-cloud'
model_to_use = 'qwen3-vl:235b'

apikey_ollam='6afb7604323e49b499631fe16e02c388.VQ7czOp3yqAKsw8m9uJcAFX1'
try:
    # Initialize Ollama client with host and authorization header
    client = Client(
        host='https://ollama.com',
        headers={'Authorization': 'Bearer ' + apikey_ollam}
    )

    full_summary = ""
    print(f"Calling Ollama API with model: {model_to_use}...")
    for part in client.chat(model=model_to_use, messages=overall_messages, stream=True):
        content = part.get('message', {}).get('content', '')
        if content:
            full_summary += content
            print(content, end='', flush=True)
    print("\n--- Summary Complete ---")
    print(full_summary)
    full_summary =f"Calling Ollama API with model: {model_to_use}..."+full_summary
    #write fullsummary to " r'/content/drive/MyDrive/LINE_mail_backup2008Gmail/fullsummary_ollam.txt'"
    with open(r'/content/drive/MyDrive/LINE_mail_backup2008Gmail/fullsummary_ollam.txt', 'w') as f:
        f.write(full_summary)
        print("fullsummary_ollam.txt written successfully.")

except ResponseError as e:
    if e.status_code == 401:
        print(f"Error: 401 Unauthorized. Please check your Ollama Cloud API endpoint ('https://ollama.com') and your API key ('{apikey_ollam}').")
    else:
        print(f"An Ollama API error occurred: Status Code {e.status_code} - {e.message}")
except requests.exceptions.RequestException as e:
    print(f"A network or request error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Calling Ollama API with model: qwen3-vl:235b...
# 近5日股票市場個股及產業訊息及新聞摘要

## 市場整體動態
- 美國財政問題與通膨擔憂引發10~30年期美債殖利率攀升，形成熊陡(Bear Steepening)曲線
- AI題材降溫，博通財報未能釋疑，導致科技、半導體類股重挫拉回，上週五美股指數全面下挫
- 台股雖有聯發科、智邦等權值股逆勢上漲，仍不敵台積電為首的多數權值下跌拖累
- 加權指數失守28,000點整數關卡，但整體格局仍堅守月線(MA20)之上，長天期均線維持多頭排列

## 重點產業/個股動向
1. **AI相關產業**：
   - PCB、電力題材股輪動轉強：新復興(4909)、晟鈦(3229)、燿華(2367)、亞力(1514)、華城(1519)等
   - SpaceX供應鏈與低軌衛星概念股：元晶(6443)、事欣科(4916)、啟碁(6285)、昇達科(3491)等
   - 矽光子：聯電取得iSiPP300 12吋矽光子製程技術，相關股：聯電(2303)、波若威(3163)等
   - 重電產業：AI資料中心與電廠建設大量湧現，大型變壓器需求暴衝，華城(1519)在手訂單達120億元

2. **半導體產業**：
   - 台積電2H26擴大釋出CoW訂單，封測廠「類CoWoS」技術崛起
   - 2026年AI晶片版圖將由GPU擴展至ASIC，雲端服務商加速自研ASIC
   - 博通(AVGO US)發布財報，揭露Anthropic為第四大客戶，近兩季下單210億美元

3. **其他產業動向**：
   - AI需求帶動全球PCB產能重組，中台日韓全面「換地圖」，台廠啟動「中國加一」
   - 華城(1519)受惠AI資料中心建置潮，相關變壓器在手訂單約120億元，AI營收占比將由5%提升至10%
   - 聯發科傳出獲Google TPU追單，有望挹注明後年獲利逾二個股本

## 重要公司資訊
- **華城(1519)**：北美國外銷占比已超50%，2026年外銷比重預計達60%以上，AI資料中心占比將超10%
- **台燿(6274)**：AI GPU/ASIC訂單強勁，泰國廠產線持續開出，4Q25營收達成率達68.6%
- **博通**：AI ASIC業務積壓訂單達730億美元